In [1]:
import brian2 as b
from brian2.units import amp, cm, ms, mV, ohm, pA, uF, um, uS, nS, mvolt, Hz
from dendrify import Dendrite, NeuronModel, Soma

# Brian params
INTEGRATION_METHOD = 'euler'
b.prefs.codegen.target = 'numpy'

# Simulation params
G_AMPA_DIST = 1*nS
G_AMPA_PROX = 2*nS
TAU_AMPA = 5*ms

In [2]:
%%timeit -r 10 -n 1

b.start_scope()

N = 10

# create compartments
soma = Soma('soma', model='leakyIF', length=25*um, diameter=25*um )
trunk = Dendrite('trunk', length=100*um, diameter=2.5*um)
prox = Dendrite('prox', length=100*um, diameter=1*um)
dist = Dendrite('dist', length=100*um, diameter=0.5*um)

# add synapses
dist.synapse('AMPA', pre='X', g=G_AMPA_DIST,  t_decay=TAU_AMPA)
prox.synapse('AMPA', pre='Y', g=G_AMPA_PROX,  t_decay=TAU_AMPA)

# add dSpikes
trunk.dspikes('Na', threshold=-40*mV, g_rise=34*nS, g_fall=27.2*nS)
prox.dspikes('Na', threshold=-40*mV, g_rise=15.3*nS, g_fall=12.24*nS)
dist.dspikes('Na', threshold=-40*mV, g_rise=7*nS, g_fall=5.6*nS)

# merge compartments into a  neuron model and set its basic properties
edges = [(soma, trunk, 15*nS), (trunk, prox, 10*nS), (prox, dist, 4*nS)]
model = NeuronModel(edges, cm=1*uF/(cm**2), gl=40*uS/(cm**2),
                    r_axial=150*ohm*cm, scale_factor=2.8, spine_factor=1.5,
                    v_rest=-60*mV)

# set Na spikes properties
model.dspike_properties('Na', tau_rise=0.6*ms, tau_fall=1.2*ms,
                            refractory=5*ms, offset_fall=0.2*ms)

# create a Brian NeuronGroup and link it to the neuron model
group = b.NeuronGroup(N, model=model.equations,
                          method=INTEGRATION_METHOD,
                          threshold='V_soma > -50*mV',
                          reset='V_soma = -60*mV',
                          refractory=4*ms,
                          namespace=model.parameters,
                          events=model.events)
group.V_soma = -60. * mvolt
group.V_trunk = -60. * mvolt
group.V_prox = -60. * mvolt
group.V_dist = -60. * mvolt
group.allow_I_Na_trunk = True 
group.allow_I_Kn_trunk = False
group.allow_I_Na_prox = True 
group.allow_I_Kn_prox = False
group.allow_I_Na_dist = True 
group.allow_I_Kn_dist = False
group.run_on_event('activate_I_Na_trunk', 'g_Na_trunk += g_Na_trunk_max; allow_I_Na_trunk=False; allow_I_Kn_trunk=True; timer_Na_trunk = t') 
group.run_on_event('activate_I_Kn_trunk', 'g_Kn_trunk += g_Kn_trunk_max; allow_I_Kn_trunk=False; allow_I_Na_trunk=True')
group.run_on_event('activate_I_Na_prox', 'g_Na_prox += g_Na_prox_max; allow_I_Na_prox=False; allow_I_Kn_prox=True; timer_Na_prox = t') 
group.run_on_event('activate_I_Kn_prox', 'g_Kn_prox += g_Kn_prox_max; allow_I_Kn_prox=False; allow_I_Na_prox=True')
group.run_on_event('activate_I_Na_dist', 'g_Na_dist += g_Na_dist_max; allow_I_Na_dist=False; allow_I_Kn_dist=True; timer_Na_dist = t') 
group.run_on_event('activate_I_Kn_dist', 'g_Kn_dist += g_Kn_dist_max; allow_I_Kn_dist=False; allow_I_Na_dist=True')

# add input
Input = b.PoissonGroup(N*2, 50*Hz)
Input_dist = Input[0:N]
Input_prox = Input[N:]

S1 = b.Synapses(Input_dist, group, on_pre='s_AMPA_X_dist += 1')
S1.connect(j='i')

S2 = b.Synapses(Input_prox, group, on_pre='s_AMPA_Y_prox += 1')
S2.connect(j='i')

b.run(1000*ms)


2.57 s ± 68.7 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [3]:
%%timeit -r 10 -n 1

b.start_scope()

N = 10**2

# create compartments
soma = Soma('soma', model='leakyIF', length=25*um, diameter=25*um )
trunk = Dendrite('trunk', length=100*um, diameter=2.5*um)
prox = Dendrite('prox', length=100*um, diameter=1*um)
dist = Dendrite('dist', length=100*um, diameter=0.5*um)

# add synapses
dist.synapse('AMPA', pre='X', g=G_AMPA_DIST,  t_decay=TAU_AMPA)
prox.synapse('AMPA', pre='Y', g=G_AMPA_PROX,  t_decay=TAU_AMPA)

# add dSpikes
trunk.dspikes('Na', threshold=-40*mV, g_rise=34*nS, g_fall=27.2*nS)
prox.dspikes('Na', threshold=-40*mV, g_rise=15.3*nS, g_fall=12.24*nS)
dist.dspikes('Na', threshold=-40*mV, g_rise=7*nS, g_fall=5.6*nS)

# merge compartments into a  neuron model and set its basic properties
edges = [(soma, trunk, 15*nS), (trunk, prox, 10*nS), (prox, dist, 4*nS)]
model = NeuronModel(edges, cm=1*uF/(cm**2), gl=40*uS/(cm**2),
                    r_axial=150*ohm*cm, scale_factor=2.8, spine_factor=1.5,
                    v_rest=-60*mV)

# set Na spikes properties
model.dspike_properties('Na', tau_rise=0.6*ms, tau_fall=1.2*ms,
                            refractory=5*ms, offset_fall=0.2*ms)

# create a Brian NeuronGroup and link it to the neuron model
group = b.NeuronGroup(N, model=model.equations,
                          method=INTEGRATION_METHOD,
                          threshold='V_soma > -50*mV',
                          reset='V_soma = -60*mV',
                          refractory=4*ms,
                          namespace=model.parameters,
                          events=model.events)
group.V_soma = -60. * mvolt
group.V_trunk = -60. * mvolt
group.V_prox = -60. * mvolt
group.V_dist = -60. * mvolt
group.allow_I_Na_trunk = True 
group.allow_I_Kn_trunk = False
group.allow_I_Na_prox = True 
group.allow_I_Kn_prox = False
group.allow_I_Na_dist = True 
group.allow_I_Kn_dist = False
group.run_on_event('activate_I_Na_trunk', 'g_Na_trunk += g_Na_trunk_max; allow_I_Na_trunk=False; allow_I_Kn_trunk=True; timer_Na_trunk = t') 
group.run_on_event('activate_I_Kn_trunk', 'g_Kn_trunk += g_Kn_trunk_max; allow_I_Kn_trunk=False; allow_I_Na_trunk=True')
group.run_on_event('activate_I_Na_prox', 'g_Na_prox += g_Na_prox_max; allow_I_Na_prox=False; allow_I_Kn_prox=True; timer_Na_prox = t') 
group.run_on_event('activate_I_Kn_prox', 'g_Kn_prox += g_Kn_prox_max; allow_I_Kn_prox=False; allow_I_Na_prox=True')
group.run_on_event('activate_I_Na_dist', 'g_Na_dist += g_Na_dist_max; allow_I_Na_dist=False; allow_I_Kn_dist=True; timer_Na_dist = t') 
group.run_on_event('activate_I_Kn_dist', 'g_Kn_dist += g_Kn_dist_max; allow_I_Kn_dist=False; allow_I_Na_dist=True')

# add input
Input = b.PoissonGroup(N*2, 50*Hz)
Input_dist = Input[0:N]
Input_prox = Input[N:]

S1 = b.Synapses(Input_dist, group, on_pre='s_AMPA_X_dist += 1')
S1.connect(j='i')

S2 = b.Synapses(Input_prox, group, on_pre='s_AMPA_Y_prox += 1')
S2.connect(j='i')

b.run(1000*ms)


2.77 s ± 73 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [4]:
%%timeit -r 10 -n 1

b.start_scope()

N = 10**3

# create compartments
soma = Soma('soma', model='leakyIF', length=25*um, diameter=25*um )
trunk = Dendrite('trunk', length=100*um, diameter=2.5*um)
prox = Dendrite('prox', length=100*um, diameter=1*um)
dist = Dendrite('dist', length=100*um, diameter=0.5*um)

# add synapses
dist.synapse('AMPA', pre='X', g=G_AMPA_DIST,  t_decay=TAU_AMPA)
prox.synapse('AMPA', pre='Y', g=G_AMPA_PROX,  t_decay=TAU_AMPA)

# add dSpikes
trunk.dspikes('Na', threshold=-40*mV, g_rise=34*nS, g_fall=27.2*nS)
prox.dspikes('Na', threshold=-40*mV, g_rise=15.3*nS, g_fall=12.24*nS)
dist.dspikes('Na', threshold=-40*mV, g_rise=7*nS, g_fall=5.6*nS)

# merge compartments into a  neuron model and set its basic properties
edges = [(soma, trunk, 15*nS), (trunk, prox, 10*nS), (prox, dist, 4*nS)]
model = NeuronModel(edges, cm=1*uF/(cm**2), gl=40*uS/(cm**2),
                    r_axial=150*ohm*cm, scale_factor=2.8, spine_factor=1.5,
                    v_rest=-60*mV)

# set Na spikes properties
model.dspike_properties('Na', tau_rise=0.6*ms, tau_fall=1.2*ms,
                            refractory=5*ms, offset_fall=0.2*ms)

# create a Brian NeuronGroup and link it to the neuron model
group = b.NeuronGroup(N, model=model.equations,
                          method=INTEGRATION_METHOD,
                          threshold='V_soma > -50*mV',
                          reset='V_soma = -60*mV',
                          refractory=4*ms,
                          namespace=model.parameters,
                          events=model.events)
group.V_soma = -60. * mvolt
group.V_trunk = -60. * mvolt
group.V_prox = -60. * mvolt
group.V_dist = -60. * mvolt
group.allow_I_Na_trunk = True 
group.allow_I_Kn_trunk = False
group.allow_I_Na_prox = True 
group.allow_I_Kn_prox = False
group.allow_I_Na_dist = True 
group.allow_I_Kn_dist = False
group.run_on_event('activate_I_Na_trunk', 'g_Na_trunk += g_Na_trunk_max; allow_I_Na_trunk=False; allow_I_Kn_trunk=True; timer_Na_trunk = t') 
group.run_on_event('activate_I_Kn_trunk', 'g_Kn_trunk += g_Kn_trunk_max; allow_I_Kn_trunk=False; allow_I_Na_trunk=True')
group.run_on_event('activate_I_Na_prox', 'g_Na_prox += g_Na_prox_max; allow_I_Na_prox=False; allow_I_Kn_prox=True; timer_Na_prox = t') 
group.run_on_event('activate_I_Kn_prox', 'g_Kn_prox += g_Kn_prox_max; allow_I_Kn_prox=False; allow_I_Na_prox=True')
group.run_on_event('activate_I_Na_dist', 'g_Na_dist += g_Na_dist_max; allow_I_Na_dist=False; allow_I_Kn_dist=True; timer_Na_dist = t') 
group.run_on_event('activate_I_Kn_dist', 'g_Kn_dist += g_Kn_dist_max; allow_I_Kn_dist=False; allow_I_Na_dist=True')

# add input
Input = b.PoissonGroup(N*2, 50*Hz)
Input_dist = Input[0:N]
Input_prox = Input[N:]

S1 = b.Synapses(Input_dist, group, on_pre='s_AMPA_X_dist += 1')
S1.connect(j='i')

S2 = b.Synapses(Input_prox, group, on_pre='s_AMPA_Y_prox += 1')
S2.connect(j='i')

b.run(1000*ms)


3.65 s ± 38.9 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [5]:
%%timeit -r 10 -n 1

b.start_scope()

N = 10**4

# create compartments
soma = Soma('soma', model='leakyIF', length=25*um, diameter=25*um )
trunk = Dendrite('trunk', length=100*um, diameter=2.5*um)
prox = Dendrite('prox', length=100*um, diameter=1*um)
dist = Dendrite('dist', length=100*um, diameter=0.5*um)

# add synapses
dist.synapse('AMPA', pre='X', g=G_AMPA_DIST,  t_decay=TAU_AMPA)
prox.synapse('AMPA', pre='Y', g=G_AMPA_PROX,  t_decay=TAU_AMPA)

# add dSpikes
trunk.dspikes('Na', threshold=-40*mV, g_rise=34*nS, g_fall=27.2*nS)
prox.dspikes('Na', threshold=-40*mV, g_rise=15.3*nS, g_fall=12.24*nS)
dist.dspikes('Na', threshold=-40*mV, g_rise=7*nS, g_fall=5.6*nS)

# merge compartments into a  neuron model and set its basic properties
edges = [(soma, trunk, 15*nS), (trunk, prox, 10*nS), (prox, dist, 4*nS)]
model = NeuronModel(edges, cm=1*uF/(cm**2), gl=40*uS/(cm**2),
                    r_axial=150*ohm*cm, scale_factor=2.8, spine_factor=1.5,
                    v_rest=-60*mV)

# set Na spikes properties
model.dspike_properties('Na', tau_rise=0.6*ms, tau_fall=1.2*ms,
                            refractory=5*ms, offset_fall=0.2*ms)

# create a Brian NeuronGroup and link it to the neuron model
group = b.NeuronGroup(N, model=model.equations,
                          method=INTEGRATION_METHOD,
                          threshold='V_soma > -50*mV',
                          reset='V_soma = -60*mV',
                          refractory=4*ms,
                          namespace=model.parameters,
                          events=model.events)
group.V_soma = -60. * mvolt
group.V_trunk = -60. * mvolt
group.V_prox = -60. * mvolt
group.V_dist = -60. * mvolt
group.allow_I_Na_trunk = True 
group.allow_I_Kn_trunk = False
group.allow_I_Na_prox = True 
group.allow_I_Kn_prox = False
group.allow_I_Na_dist = True 
group.allow_I_Kn_dist = False
group.run_on_event('activate_I_Na_trunk', 'g_Na_trunk += g_Na_trunk_max; allow_I_Na_trunk=False; allow_I_Kn_trunk=True; timer_Na_trunk = t') 
group.run_on_event('activate_I_Kn_trunk', 'g_Kn_trunk += g_Kn_trunk_max; allow_I_Kn_trunk=False; allow_I_Na_trunk=True')
group.run_on_event('activate_I_Na_prox', 'g_Na_prox += g_Na_prox_max; allow_I_Na_prox=False; allow_I_Kn_prox=True; timer_Na_prox = t') 
group.run_on_event('activate_I_Kn_prox', 'g_Kn_prox += g_Kn_prox_max; allow_I_Kn_prox=False; allow_I_Na_prox=True')
group.run_on_event('activate_I_Na_dist', 'g_Na_dist += g_Na_dist_max; allow_I_Na_dist=False; allow_I_Kn_dist=True; timer_Na_dist = t') 
group.run_on_event('activate_I_Kn_dist', 'g_Kn_dist += g_Kn_dist_max; allow_I_Kn_dist=False; allow_I_Na_dist=True')

# add input
Input = b.PoissonGroup(N*2, 50*Hz)
Input_dist = Input[0:N]
Input_prox = Input[N:]

S1 = b.Synapses(Input_dist, group, on_pre='s_AMPA_X_dist += 1')
S1.connect(j='i')

S2 = b.Synapses(Input_prox, group, on_pre='s_AMPA_Y_prox += 1')
S2.connect(j='i')

b.run(1000*ms)


9.87 s ± 241 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [6]:
%%timeit -r 10 -n 1

b.start_scope()

N = 10**5

# create compartments
soma = Soma('soma', model='leakyIF', length=25*um, diameter=25*um )
trunk = Dendrite('trunk', length=100*um, diameter=2.5*um)
prox = Dendrite('prox', length=100*um, diameter=1*um)
dist = Dendrite('dist', length=100*um, diameter=0.5*um)

# add synapses
dist.synapse('AMPA', pre='X', g=G_AMPA_DIST,  t_decay=TAU_AMPA)
prox.synapse('AMPA', pre='Y', g=G_AMPA_PROX,  t_decay=TAU_AMPA)

# add dSpikes
trunk.dspikes('Na', threshold=-40*mV, g_rise=34*nS, g_fall=27.2*nS)
prox.dspikes('Na', threshold=-40*mV, g_rise=15.3*nS, g_fall=12.24*nS)
dist.dspikes('Na', threshold=-40*mV, g_rise=7*nS, g_fall=5.6*nS)

# merge compartments into a  neuron model and set its basic properties
edges = [(soma, trunk, 15*nS), (trunk, prox, 10*nS), (prox, dist, 4*nS)]
model = NeuronModel(edges, cm=1*uF/(cm**2), gl=40*uS/(cm**2),
                    r_axial=150*ohm*cm, scale_factor=2.8, spine_factor=1.5,
                    v_rest=-60*mV)

# set Na spikes properties
model.dspike_properties('Na', tau_rise=0.6*ms, tau_fall=1.2*ms,
                            refractory=5*ms, offset_fall=0.2*ms)

# create a Brian NeuronGroup and link it to the neuron model
group = b.NeuronGroup(N, model=model.equations,
                          method=INTEGRATION_METHOD,
                          threshold='V_soma > -50*mV',
                          reset='V_soma = -60*mV',
                          refractory=4*ms,
                          namespace=model.parameters,
                          events=model.events)
group.V_soma = -60. * mvolt
group.V_trunk = -60. * mvolt
group.V_prox = -60. * mvolt
group.V_dist = -60. * mvolt
group.allow_I_Na_trunk = True 
group.allow_I_Kn_trunk = False
group.allow_I_Na_prox = True 
group.allow_I_Kn_prox = False
group.allow_I_Na_dist = True 
group.allow_I_Kn_dist = False
group.run_on_event('activate_I_Na_trunk', 'g_Na_trunk += g_Na_trunk_max; allow_I_Na_trunk=False; allow_I_Kn_trunk=True; timer_Na_trunk = t') 
group.run_on_event('activate_I_Kn_trunk', 'g_Kn_trunk += g_Kn_trunk_max; allow_I_Kn_trunk=False; allow_I_Na_trunk=True')
group.run_on_event('activate_I_Na_prox', 'g_Na_prox += g_Na_prox_max; allow_I_Na_prox=False; allow_I_Kn_prox=True; timer_Na_prox = t') 
group.run_on_event('activate_I_Kn_prox', 'g_Kn_prox += g_Kn_prox_max; allow_I_Kn_prox=False; allow_I_Na_prox=True')
group.run_on_event('activate_I_Na_dist', 'g_Na_dist += g_Na_dist_max; allow_I_Na_dist=False; allow_I_Kn_dist=True; timer_Na_dist = t') 
group.run_on_event('activate_I_Kn_dist', 'g_Kn_dist += g_Kn_dist_max; allow_I_Kn_dist=False; allow_I_Na_dist=True')

# add input
Input = b.PoissonGroup(N*2, 50*Hz)
Input_dist = Input[0:N]
Input_prox = Input[N:]

S1 = b.Synapses(Input_dist, group, on_pre='s_AMPA_X_dist += 1')
S1.connect(j='i')

S2 = b.Synapses(Input_prox, group, on_pre='s_AMPA_Y_prox += 1')
S2.connect(j='i')

b.run(1000*ms)


1min 33s ± 678 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
